## Transfer learning : Feature Extraction

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import optimizers
from keras import losses
from keras import activations
from keras import callbacks
from keras import utils
from keras import metrics
from keras import applications


from keras import datasets

In [3]:
!nvidia-smi

Tue Feb  4 09:39:26 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650        Off | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P0               5W /  50W |      6MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### *Get the data*

In [4]:
# import zipfile

# # Download data
# !wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# # Unzip the  downloaded file
# zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
# zip_ref.extractall()
# zip_ref.close()

In [5]:
# How many images in each folder?
import os

# Walk through 10 percent data directories and list the number of files
for dirpath, dirname, filename in os.walk("10_food_classes_10_percent"):
    print(f"There'are {len(dirname)} directories and {len(filename)} images in '{dirpath}'")


There'are 2 directories and 0 images in '10_food_classes_10_percent'
There'are 10 directories and 0 images in '10_food_classes_10_percent/test'
There'are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'
There'are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'
There'are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'
There'are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'
There'are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'
There'are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'
There'are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'
There'are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'
There'are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'
There'are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi

### *Creating data loaders (preparing the data)*

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Image_shape  = (224, 224)
Batch_size = 32
c_mode = "categorical"

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.0)
test_datagen = ImageDataGenerator(rescale=1/255.0)

print("Training Images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                        target_size=Image_shape,
                                                        batch_size=Batch_size,
                                                        class_mode=c_mode)

print("Testing Image:")
test_data = train_datagen.flow_from_directory(test_dir,
                                            target_size=Image_shape,
                                            batch_size=Batch_size,
                                            class_mode=c_mode)







Training Images:
Found 750 images belonging to 10 classes.
Testing Image:
Found 2500 images belonging to 10 classes.


In [7]:
# Create TensorBoard callback (functionized because need to create a new one for each model)

import datetime

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/"  + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving Tensorboard log files to: {log_dir}")
    return tensorboard_callback







In [8]:
import tensorflow as tf
import tensorflow_hub as hub

import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))




Num GPUs Available:  1


In [9]:
# Resnet 50 v2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# original: EfficientNet50 feature vector (version 1)
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# New: EfficientNetB0 feature vector (version 2)
efficientnet_url_new = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"




In [ ]:
import tensorflow_hub as hub

def create_mode(model_url, num_classes=10):
    """
    Take a tensorflow Hub URL and creates a Keras Sequential model with it
    
    Args: 
        model_url: A TensorflowHub feature extraction URL
        num_classes: Number of output neurons in output layer,
        should be equal to number of target classes, default 10,
        
    Returns:
        An uncompiled Keras Sequential model with model_url as feature 
         extractor layer and Dense output layer with num_classes outputs """
    
    # Download the pretrained model and save it as Keras Layer
    feature_extractor_layer = hub.KerasLayer(model_url,
                                             trainable=False,   # frozen the underlining patterns
                                             name="feature_extraction_layer",
                                             input_shape= Image_shape+(3,)) 
    

    # Create out own model
    model = keras.Sequential([
        feature_extractor_layer,
        layers.Dense(num_classes, activation=activations.softmax, name="output_layer")
    ])

    return model


# Create model
resnet_model = create_mode(resnet_url, num_classes=train_data_10_percent.num_classes)

# Compile model
resnet_model.compile(loss=losses.categorical_crossentropy,
                     optimizer=optimizers.Adam(),
                     metric=[metrics.categorical_accuracy])


# Fit the model
resnet_history = resnet_model.fit(train_data_10_percent,
                                  epochs=5,
                                  steps_per_epoch=len(train_data_10_percent),
                                  validation_data=test_data,
                                  validation_steps=len(test_data),
                                  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",         # Save the experiment log here, directory
                                                                         experiment_name="resnet50v2")])    # Name of the experiment





### *EfficientNetV250*

In [12]:
# 1.Create base model with tf.keras.applications
base_model = applications.EfficientNetV2B0(include_top=False)

# 2. Freeze the base model (so the pre-learned patterns remain)
base_model.trainable = False

# 3. Create inputs into our model
inputs = layers.Input(shape=(224, 224, 3), name="input_layer")

# 4. If using ResNet50V2 you will need to normalize inputs (you don't have to for EfficientNet(s))
# x = keras.preprocessing.Rescaling(1./255)(x)

# 5. Pass the inputs to the base_model (note: using tf.keras.applications, EfficientNet inputs don't have to be normalized)
x = base_model(inputs)

# check the shape of the output of the base model
print("Shape after base_model:", x.shape) # x is the output of the EfficientNetB0 base model

# 6. Average pool the outputs of the base model (aggregate all the most important information, reduce number of computations)
x = layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print("Output shape after GlobalAveragePooling2D:", x.shape)

# 7. Create the output activation layer
outputs = layers.Dense(10, activation=activations.softmax, name="output_layer")(x)

# 8. Combine the inputs with the outputs into a model
model = keras.Model(inputs, outputs)

# 9. Compile the model
model.compile(loss=losses.categorical_crossentropy,
              optimizer=optimizers.Adam(),
              metrics=[metrics.categorical_accuracy])




Shape after base_model: (None, 7, 7, 1280)
Output shape after GlobalAveragePooling2D: (None, 1280)


In [15]:
# 10. Fit the model
efficientnet_history = model.fit(train_data_10_percent,
                                 epochs=5,
                                 steps_per_epoch=len(train_data_10_percent),
                                 validation_data=test_data,
                                 validation_steps=int(0.25 * len(test_data)),
                                #  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                #                                         experiment_name="efficientnetb0")]
                                )

Epoch 1/5


E0000 00:00:1738637614.873580    8970 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
E0000 00:00:1738637615.967794    8970 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2025-02-04 09:53:36.017717: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at xla_ops.cc:577 : FAILED_PRECONDITION: DNN library initialization failed. Look at the er

FailedPreconditionError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/lovecrush/miniconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/lovecrush/miniconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/lovecrush/miniconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/lovecrush/miniconda3/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/lovecrush/miniconda3/lib/python3.12/asyncio/base_events.py", line 639, in run_forever

  File "/home/lovecrush/miniconda3/lib/python3.12/asyncio/base_events.py", line 1985, in _run_once

  File "/home/lovecrush/miniconda3/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/home/lo